<a href="https://colab.research.google.com/github/annmary25/Text-Summarization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
from transformers import pipeline

In [55]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [56]:
import requests
from bs4 import BeautifulSoup

In [57]:
URL = "https://en.wikipedia.org/wiki/Data_science"
response = requests.get(URL)

In [58]:
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [59]:
ARTICLE

'Data science \n Data science is an interdisciplinary academic field [1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" in order to "understand and analyse actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turi

**Text Summarization**

In [60]:
max_chunk = 500

In [61]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [62]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])


0


In [63]:
len(chunks)

3

In [64]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


In [65]:
' '.join([summ['summary_text'] for summ in res])

'Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data. Data science is multifaceted and can be described as a science, research paradigm, a research method, a discipline, a workflow, and a profession. In 1962, John Tukey described a field he called "data analysis", which resembles modern data science. In 1985, C.F. Jeff Wu used the term "data science" for the first time as an alternative name for statistics. In 1998, Hayashi Chikio argued for data science as a new, interdisciplinary concept, with three aspects: data design, collection, and analysis. In 2012, technologists Thomas H. Davenport and DJ Patil declared "Data Scientist: The Sexiest Job of the 21st Century" There is still no consensus on the definition of data science. Big data is a related marketing term. Data scientists are responsible 

**ABSTRACTIVE TEXT SUMMARIZATION**

In [66]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [67]:
tokens = tokenizer(ARTICLE, max_length=120, truncation=True, padding="longest", return_tensors="pt")

In [68]:
summary = model.generate(**tokens)

In [69]:
summary

tensor([[    2,     0, 30383,  2866,    16,    41,  3222, 32231,  5286,   882,
            14,  2939,  6732,     6,  6441, 11730,     6,  6441,  6448,     6,
          5588,     6, 16964,     8,  1743,     7, 14660,    50, 39831, 41511,
          2655,     8,  8418,    31, 28269,     6, 16697,     6,     8,   542,
         25384,  4075,   414,     4,  5423,  2866,    67, 33752, 11170,  2655,
            31,     5,  7482,  2502, 11170,    36,   242,     4,   571,     4,
          1632, 17874,     6,   335,   806,     6,     8,  6150,    43,     2]])

In [70]:
tokenizer.decode(summary[0])

'</s><s>Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data. Data science also integrates domain knowledge from the underlying application domain (e.g. natural sciences, information technology, and medicine)</s>'